# ETL Pipeline

This notebook is part of my portfolio to demonstrate my skills in _data analytics_ and _data science_. Here I can demonstrate all the knowledge acquired in my career as a mechatronics engineer and the various courses in the work area such as Python SQL and Power BI.

They are used:

- SQL server 2022
- SQL Server Management Studio (SSMS) 20.2
- Power BI 2.138
- Python 3.12.7

In [ ]:
# import the libraries to be use
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import psycopg2
from pathlib import Path

## Extracting data from various source

### Extracting data from csv file using Pandas

In [ ]:
#Using library os, get the information from the files in the folder and print the folder and files names 
import os
for dirfolder, _, filenames in os.walk('/data'):
    for name in filenames:
        print(os.path.join(dirfolder, name))

In [ ]:
#Open the file CSV using pandas and save the data in a dataframe
dataframe = pd.read_csv('../dataset.csv')

In [ ]:
# View the dataframe column headers
dataframe.head()
# The structure of the dataframe
dataframe.shape

In [ ]:
# count the number of null values in the data set
dataframe.isnull().sum()

.parquet

In [ ]:
try:
    # for pyhton file .py
    PATH = Path(__file__).resolve().parent.parent
except NameError:
    # For Jupyter Notebook
    PATH = Path().resolve().parent.parent

with open(PATH / "config.json", "r") as f:
    VARIABLES = json.load(f)

# path of the folder with .parquet
folder_path = Path(VARIABLES["file_path_sampled_box_builder_dataset"])
feature_frame = pd.read_csv(folder_path)

## Connection to Database in SQL

In [ ]:
try:
    database = "collection_tk"
    global connection
    connection = psycopg2.connect(
        host='localhost',
        user='postgres',
        password = '1234',
        database = database
    )
    print("Correct connection to database")
    #return True

except Exception as ex:
    print(f" Error connecting to database: {ex}")
    #return False

### Insert in the column in the database

In [ ]:
table = "tiktok_links_v1"
database = "collection_tk"
datos = " "
#for psycopg2 the table is necessary to be in lowercase, even declarad in the DB

try:
    cursor=connection.cursor()
    encabezados = datos.columns
    
    #datos['ID_global'].iloc[1] #to get a specific data based in index
    for i,_ in enumerate(datos.iterrows()):
        row = datos.iloc[i]
        
        value = "("
        for j,_ in enumerate(encabezados):
            if isinstance(row[encabezados[j]],str):
                value += f"'{row[encabezados[j]]}',"
            else:
                value += f"{str(row[encabezados[j]])},"

            #value = value+str(row[encabezados[j]])+","
        value = value[:-1]+")"
        
        try:
            cursor.execute(f"INSERT INTO {table} VALUES {value}")
            print("executed: "+f"INSERT INTO {table} VALUES {value}")
        except Exception as ex:
            print(f"error in the transaction: {ex}")

    connection.commit()
    print("Upload Correctly")
    
except Exception as error:
    print(f"Error modifying the table: {error}")
    connection.rollback()  # revert the transaction if there is a error
finally:
    cursor.close()
    connection.close()

### Do a Querry to the database

In [ ]:
querry = "SELECT * FROM tiktok_links_v1"
try:
    cursor = connection.cursor()
    cursor.execute(querry)
    
    #if a querry:
    if querry.strip().upper().startswith("SELECT"):
        result= cursor.fetchall()
        print("Querry successful")
    
    #if diferent to a Querry
    connection.commit()
    print("Changes applied")
    

except Exception as ex:
    print(f"Error al conectar a la base de datos: {ex}")
    connection.rollback()  # revert the transaction if there is a error
finally:
    cursor.close()
    connection.close()

## Get data from a Json and used to get data from CSV

In [ ]:
import json
from pathlib import Path
file = "links_v1"
#A JSON is used to save all the path from the desktop
#We obtain the main path of the project
base_dir = Path().resolve()
#path(): returns a object type path
#.resolve: resolve posible issues eith the path
with open(base_dir / "config.json", "r") as f:
    paths = json.load(f)
    #Convert a file json in a dictonary
#Creacion de DataFrame en pandas
datos= pd.read_csv(paths[file])